In [1]:
# === cell 1: imports & paths ===
import os, sys, json, math, glob, random
import pandas as pd
import numpy as np
from PIL import Image
from pathlib import Path

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models

from sklearn.metrics import f1_score, average_precision_score, roc_auc_score
from sklearn.model_selection import train_test_split

CSV_PATH = Path("/kaggle/input/annonations/annotations_fitzpatrick17k.csv")
IMG_ROOT = Path("/kaggle/input/fitzpatrick17k-original/finalfitz17k")

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)


In [2]:
# === cell 2: read CSV & detect label columns ===
df = pd.read_csv(CSV_PATH)

# Clean up stray index column if present
if "Unnamed: 0" in df.columns:
    df = df.drop(columns=["Unnamed: 0"])

# Image identifier column
assert "ImageID" in df.columns, "Expected column 'ImageID' in the CSV."

# Detect 0/1 label columns automatically
def is_binary_col(s: pd.Series) -> bool:
    vals = pd.Series(s.dropna().unique())
    try:
        vals = vals.astype(int)
    except:
        vals = vals.astype(str)
    return set(vals.astype(str)) <= set(["0","1"])

label_cols = [c for c in df.columns if c != "ImageID" and is_binary_col(df[c])]
assert len(label_cols) > 0, "No binary label columns detected!"

print(f"Detected {len(label_cols)} concern labels.")
print(label_cols[:15], " ...")

# Convert labels to int
df[label_cols] = df[label_cols].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)


Detected 49 concern labels.
['Vesicle', 'Papule', 'Macule', 'Plaque', 'Abscess', 'Pustule', 'Bulla', 'Patch', 'Nodule', 'Ulcer', 'Crust', 'Erosion', 'Excoriation', 'Atrophy', 'Exudate']  ...


In [3]:
# === cell 3: index all images under IMG_ROOT ===
# Build a map from basename (without dirs) to full path
all_img_paths = []
for ext in ("*.jpg","*.jpeg","*.png","*.bmp","*.webp"):
    all_img_paths += list(IMG_ROOT.rglob(ext))

path_by_name = {p.name: p for p in all_img_paths}
print(f"Indexed {len(path_by_name)} files under {IMG_ROOT}.")

# Attach full paths into df (drop rows we cannot find)
df["img_path"] = df["ImageID"].map(lambda x: path_by_name.get(str(x), None))
missing = df["img_path"].isna().sum()
if missing > 0:
    print(f"WARNING: {missing} rows have missing image files. Dropping them.")
    df = df.dropna(subset=["img_path"]).reset_index(drop=True)

len(df), df.head(2)


Indexed 16574 files under /kaggle/input/fitzpatrick17k-original/finalfitz17k.


(3688,
                                 ImageID  Vesicle  Papule  Macule  Plaque  \
 0  eb0cbb277ba6b206c5fafc66ab8c46f9.jpg        0       0       0       0   
 1  bb3d08781eb23890a9909201deed8c85.jpg        0       0       0       1   
 
    Abscess  Pustule  Bulla  Patch  Nodule  ...  Poikiloderma  Salmon  Wheal  \
 0        0        0      0      0       1  ...             0       0      0   
 1        0        0      0      0       0  ...             0       0      0   
 
    Acuminate  Burrow  Gray  Pigmented  Cyst  Do not consider this image  \
 0          0       0     0          0     0                           0   
 1          0       0     0          0     0                           0   
 
                                             img_path  
 0  /kaggle/input/fitzpatrick17k-original/finalfit...  
 1  /kaggle/input/fitzpatrick17k-original/finalfit...  
 
 [2 rows x 51 columns])

In [4]:
# === cell 4: split train/val ===
X = df["img_path"].astype(str).values
Y = df[label_cols].values

# Try iterative multilabel split; otherwise fallback to random
try:
    from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
    mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
    train_idx, val_idx = next(mskf.split(X, Y))
except Exception:
    print("iterstrat not available; using random split.")
    train_idx, val_idx = train_test_split(
        np.arange(len(X)),
        test_size=0.2,
        random_state=SEED,
        shuffle=True
    )

X_train, X_val = X[train_idx], X[val_idx]
Y_train, Y_val = Y[train_idx], Y[val_idx]
len(X_train), len(X_val)


iterstrat not available; using random split.


(2950, 738)

In [5]:
# === cell 5: dataset & transforms ===
IMG_SIZE = 384

train_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomApply([transforms.ColorJitter(brightness=0.15, contrast=0.15, saturation=0.1, hue=0.03)], p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

val_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

class FitzDataset(Dataset):
    def __init__(self, paths, labels, tfms):
        self.paths = paths
        self.labels = labels
        self.tfms = tfms
    def __len__(self):
        return len(self.paths)
    def __getitem__(self, i):
        p = self.paths[i]
        y = torch.tensor(self.labels[i], dtype=torch.float32)
        with Image.open(p).convert("RGB") as im:
            im = self.tfms(im)
        return im, y, p

train_ds = FitzDataset(X_train, Y_train, train_tfms)
val_ds   = FitzDataset(X_val,   Y_val,   val_tfms)

train_dl = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=2, pin_memory=True)
val_dl   = DataLoader(val_ds,   batch_size=16, shuffle=False, num_workers=2, pin_memory=True)

n_classes = len(label_cols)


In [6]:
# === cell 6: model, loss with class weights ===
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
in_feats = model.fc.in_features
model.fc = nn.Linear(in_feats, n_classes)
model = model.to(DEVICE)

# pos_weight = (N - P) / P per class
P = Y_train.sum(axis=0) + 1e-6
N = len(Y_train) - P
pos_weight = torch.tensor((N / P), dtype=torch.float32, device=DEVICE)

criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=5)


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 230MB/s]


In [7]:
# === cell 7: training loop ===
def sigmoid(x): return 1/(1+np.exp(-x))

def evaluate(model, loader, thresh=0.3):
    model.eval()
    all_logits, all_targets = [], []
    with torch.no_grad():
        for imgs, ys, _ in loader:
            imgs = imgs.to(DEVICE)
            ys = ys.to(DEVICE)
            logits = model(imgs)
            all_logits.append(logits.detach().cpu().numpy())
            all_targets.append(ys.detach().cpu().numpy())
    logits = np.concatenate(all_logits)
    targets = np.concatenate(all_targets)
    probs = sigmoid(logits)

    # Macro F1 at a single threshold
    preds = (probs >= thresh).astype(int)
    macro_f1 = f1_score(targets, preds, average="macro", zero_division=0)

    # Macro AUROC / mAP (handle classes with only one label carefully)
    aurocs, aps = [], []
    for c in range(targets.shape[1]):
        if len(np.unique(targets[:,c])) > 1:
            try:
                aurocs.append(roc_auc_score(targets[:,c], probs[:,c]))
            except:
                pass
            try:
                aps.append(average_precision_score(targets[:,c], probs[:,c]))
            except:
                pass
    macro_auroc = float(np.mean(aurocs)) if aurocs else 0.0
    macro_map   = float(np.mean(aps)) if aps else 0.0

    return {
        "macro_f1@{:.2f}".format(thresh): macro_f1,
        "macro_auroc": macro_auroc,
        "macro_map": macro_map
    }

EPOCHS = 200
best_val = -1
for epoch in range(1, EPOCHS+1):
    model.train()
    running_loss = 0.0
    for imgs, ys, _ in train_dl:
        imgs = imgs.to(DEVICE)
        ys = ys.to(DEVICE)
        optimizer.zero_grad()
        logits = model(imgs)
        loss = criterion(logits, ys)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * imgs.size(0)
    scheduler.step()

    val_metrics = evaluate(model, val_dl, thresh=0.3)
    val_key = "macro_f1@0.30"
    val_f1 = val_metrics[val_key]
    print(f"Epoch {epoch:02d} | loss {running_loss/len(train_ds):.4f} | "
          f"F1 {val_f1:.3f} | AUROC {val_metrics['macro_auroc']:.3f} | mAP {val_metrics['macro_map']:.3f}")

    if val_f1 > best_val:
        best_val = val_f1
        torch.save({"state_dict": model.state_dict(),
                    "label_cols": label_cols}, "best_model.pth")
        print("  ✓ saved best_model.pth")


Epoch 01 | loss 1.3196 | F1 0.092 | AUROC 0.584 | mAP 0.107
  ✓ saved best_model.pth
Epoch 02 | loss 1.2981 | F1 0.091 | AUROC 0.566 | mAP 0.094
Epoch 03 | loss 1.2286 | F1 0.094 | AUROC 0.671 | mAP 0.146
  ✓ saved best_model.pth
Epoch 04 | loss 1.1307 | F1 0.100 | AUROC 0.730 | mAP 0.167
  ✓ saved best_model.pth
Epoch 05 | loss 1.0195 | F1 0.103 | AUROC 0.725 | mAP 0.181
  ✓ saved best_model.pth
Epoch 06 | loss 0.9703 | F1 0.103 | AUROC 0.727 | mAP 0.182
Epoch 07 | loss 0.9581 | F1 0.108 | AUROC 0.745 | mAP 0.191
  ✓ saved best_model.pth
Epoch 08 | loss 0.9390 | F1 0.119 | AUROC 0.729 | mAP 0.177
  ✓ saved best_model.pth
Epoch 09 | loss 0.8901 | F1 0.120 | AUROC 0.766 | mAP 0.207
  ✓ saved best_model.pth
Epoch 10 | loss 0.8337 | F1 0.125 | AUROC 0.776 | mAP 0.210
  ✓ saved best_model.pth
Epoch 11 | loss 0.6925 | F1 0.141 | AUROC 0.781 | mAP 0.208
  ✓ saved best_model.pth
Epoch 12 | loss 0.5826 | F1 0.162 | AUROC 0.828 | mAP 0.249
  ✓ saved best_model.pth
Epoch 13 | loss 0.5133 | F1 0.

In [11]:
# Extract label names from the checkpoint and save labels.txt
import torch
from pathlib import Path

CKPT_PATH = Path("/kaggle/working/best_model.pth")   # adjust if elsewhere
OUT_PATH  = Path("/kaggle/working/labels.txt")

ckpt = torch.load(CKPT_PATH, map_location="cpu")
label_cols = ckpt.get("label_cols", None)

if label_cols is None:
    raise ValueError("Checkpoint does not contain 'label_cols'. Use method A (from CSV) or provide label order manually.")

OUT_PATH.write_text("\n".join(label_cols), encoding="utf-8")
print(f"Saved {len(label_cols)} labels to {OUT_PATH}")
print("First 10:", label_cols[:10])


Saved 49 labels to /kaggle/working/labels.txt
First 10: ['Vesicle', 'Papule', 'Macule', 'Plaque', 'Abscess', 'Pustule', 'Bulla', 'Patch', 'Nodule', 'Ulcer']


In [10]:
# === cell 8: inference helper ===
infer_tfms = val_tfms  # same as validation

def load_checkpoint(path="best_model.pth"):
    ckpt = torch.load(path, map_location=DEVICE)
    lbls = ckpt["label_cols"]
    mdl = models.resnet50(weights=None)
    mdl.fc = nn.Linear(mdl.fc.in_features, len(lbls))
    mdl.load_state_dict(ckpt["state_dict"])
    mdl.to(DEVICE).eval()
    return mdl, lbls

def predict_image(image_path, model, label_names, threshold=0.3, top_k=None):
    with Image.open(image_path).convert("RGB") as im:
        x = infer_tfms(im).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        logits = model(x)
        probs = torch.sigmoid(logits)[0].cpu().numpy()

    # All concerns with probabilities (sorted)
    idx_sorted = np.argsort(-probs)
    results = [(label_names[i], float(probs[i])) for i in idx_sorted]

    # Binary decisions at threshold
    picked = [(label_names[i], float(probs[i])) for i in range(len(label_names)) if probs[i] >= threshold]
    picked_sorted = sorted(picked, key=lambda z: -z[1])

    # Limit display if top_k set
    if top_k is not None:
        results = results[:top_k]

    return results, picked_sorted

# Example:
model, names = load_checkpoint("best_model.pth")
all_ranked, detected = predict_image("/kaggle/input/fitzpatrick17k-original/finalfitz17k/000491af8dd4d739de520e8a68be7134.jpg",
                                      model, names, threshold=0.3)
print("All concerns (sorted):")
for n,p in all_ranked: print(f"{n}: {p:.3f}")
print("\nDetected (≥0.30):")
for n,p in detected: print(f"{n}: {p:.3f}")


All concerns (sorted):
Do not consider this image: 0.881
Brown(Hyperpigmentation): 0.857
Plaque: 0.454
Papule: 0.082
Crust: 0.055
Erosion: 0.048
Yellow: 0.040
Erythema: 0.011
Scale: 0.008
Excoriation: 0.003
Purple: 0.001
White(Hypopigmentation): 0.001
Nodule: 0.001
Black: 0.001
Scar: 0.000
Patch: 0.000
Friable: 0.000
Pustule: 0.000
Dome-shaped: 0.000
Bulla: 0.000
Exophytic/Fungating: 0.000
Vesicle: 0.000
Ulcer: 0.000
Fissure: 0.000
Exudate: 0.000
Blue: 0.000
Sclerosis: 0.000
Gray: 0.000
Lichenification: 0.000
Pedunculated: 0.000
Xerosis: 0.000
Cyst: 0.000
Acuminate: 0.000
Warty/Papillomatous: 0.000
Induration: 0.000
Atrophy: 0.000
Comedo: 0.000
Telangiectasia: 0.000
Poikiloderma: 0.000
Purpura/Petechiae: 0.000
Salmon: 0.000
Wheal: 0.000
Burrow: 0.000
Umbilicated: 0.000
Macule: 0.000
Abscess: 0.000
Flat topped: 0.000
Pigmented: 0.000
Translucent: 0.000

Detected (≥0.30):
Do not consider this image: 0.881
Brown(Hyperpigmentation): 0.857
Plaque: 0.454
